<a href="https://colab.research.google.com/github/chrisseub/ML-Practice-Rep/blob/main/EGG_SIZE_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORT THE LIBRARIES

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


  IMPORT THE EGG QUALITY DATA

In [4]:
Egg_Traits = pd.read_excel('/content/main.xls')
# Now you can work with the DataFrame 'df'
print(Egg_Traits.head()) # Print the first few rows
Egg_Traits.shape


  STRAIN          AGE  EGG_WEIGHT   EGG_LENGTH  ...  Shell surface area Yolk Index Shape Index Outcome
0  BROWN  EARLY LAYER         52.5        5.75  ...           63.770823  43.902439   69.565217   Small
1  BROWN  EARLY LAYER         62.6        5.70  ...           71.623566  47.619048   75.438596   Large
2  BROWN  EARLY LAYER         61.0        5.50  ...           70.410037  45.000000   78.181818   Large
3  BROWN  EARLY LAYER         64.1        5.70  ...           72.751706  47.500000   77.192982   Large
4  BROWN  EARLY LAYER         53.9        5.60  ...           64.888161  55.000000   71.428571   Small

[5 rows x 16 columns]


(300, 16)

ANALYSING THE DATA

In [5]:
Egg_Traits.describe() #For some descriptive statistics

,EGG_WEIGHT,EGG_LENGTH,EGG_WIDTH,YOLK _ALB_WEIGHT,SHELL_WEIGTH,SHELL_THICKNESS,Haugh Unit,Shell surface area,Yolk Index,Shape Index
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,299.000000,300.000000,296.000000,300.000000
mean,63.848667,5.800800,4.340500,55.607333,7.593667,0.323033,84.539822,72.483645,48.002424,75.555584
std,6.304399,0.306651,0.291033,5.840976,0.704686,0.042691,0.744175,4.739732,4.481038,5.981744
min,49.700000,5.100000,0.300000,43.000000,5.900000,0.200000,81.801110,61.505237,21.621622,5.263158
25%,59.200000,5.587500,4.200000,51.300000,7.100000,0.290000,84.057052,69.031801,45.238095,73.098388
50%,63.650000,5.800000,4.300000,55.500000,7.500000,0.330000,84.548793,72.414210,47.619048,75.545385
75%,68.600000,6.000000,4.500000,60.350000,8.025000,0.350000,85.045960,76.083555,50.166667,78.181818
max,78.900000,7.000000,4.900000,70.200000,9.900000,0.420000,86.421165,83.442576,61.764706,89.090909
